In [12]:
import pandas as pd
import numpy as np
import os
import pybedtools as bt

In [2]:
annotation_dir = '/projects/ps-yeolab3/bay001/maps/annotations-0.05-0.1-0.05-nonredundant/'

In [5]:
# check a random rbp to make sure they don't overlap
larp = pd.read_table(os.path.join(annotation_dir,'LARP7-K562-SE.txt'))
larp_all = pd.read_table(os.path.join(annotation_dir,'LARP7-K562-SE.all.txt'))

In [10]:

df = pd.merge(larp,larp_all,how='right',on='ID')
dfx = df[pd.isnull(df).any(axis=1)] # all events that overlap with greater IJC
for col in dfx.columns:
    if '_x' in col:
        del dfx[col]
dfx.head()

,ID,GeneID_y,geneSymbol_y,chr_y,strand_y,exonStart_0base_y,exonEnd_y,upstreamES_y,upstreamEE_y,downstreamES_y,...,SJC_SAMPLE_1_y,IJC_SAMPLE_2_y,SJC_SAMPLE_2_y,IncFormLen_y,SkipFormLen_y,PValue_y,FDR_y,IncLevel1_y,IncLevel2_y,IncLevelDifference_y
18951,8337.0,ENSG00000138326.14,RPS24,chr10,+,79799961,79799983,79796951,79797062,79800372,...,"1606,1617","403,598","1753,2308",120,99,1.576517e-14,6.690737e-10,"0.07,0.077","0.159,0.176",-0.094
18952,8342.0,ENSG00000138326.14,RPS24,chr10,+,79799958,79799983,79796951,79797062,79800372,...,"1606,1617","213,326","1753,2308",123,99,5.002665e-13,1.061566e-08,"0.032,0.044","0.089,0.102",-0.058
18953,27692.0,ENSG00000132780.12,NASP,chr1,+,46072992,46074009,46072153,46072263,46078840,...,"316,293","368,373","517,414",197,99,2.855597e-09,4.039718e-05,"0.454,0.456","0.263,0.312",0.168
18954,8572.0,ENSG00000084234.12,APLP2,chr11,+,130007150,130007186,130005457,130005610,130010292,...,"92,78","44,21","120,123",134,99,1.211279e-08,1.285167e-04,"0.417,0.399","0.213,0.112",0.246
18955,25409.0,ENSG00000133112.12,TPT1,chr13,-,45914846,45914955,45914128,45914319,45915176,...,"112,80","4071,4515","57,51",197,99,1.781897e-08,1.470265e-04,"0.936,0.95","0.973,0.978",-0.032


In [13]:
import pybedtools as bt
def get_avg_inclusion_count(row):
    s1a,s1b = row['IJC_SAMPLE_1_y'].split(',')
    
    s2a,s2b = row['IJC_SAMPLE_2_y'].split(',')
    return (int(s1a)+int(s1b)+int(s2a)+int(s2b))/4.0
def make_bedtools(row,x):
    return bt.create_interval_from_list([row['chr'],row['upstreamEE_y'],row['downstreamES_y'],x,'0',row['strand']])
def return_bedtool(l):
    return bt.create_interval_from_list(l)